In [14]:
import warnings
warnings.filterwarnings('ignore')


from transformers import AlbertTokenizer, AlbertForSequenceClassification
import torch
import pandas as pd

In [2]:
query = "Как подписать контракт после отказа заказчика от заключения?"

In [3]:
save_directory = "./albert_finetuned"

In [5]:
# Load the model and tokenizer
albert_model = AlbertForSequenceClassification.from_pretrained(save_directory)
albert_tokenizer = AlbertTokenizer.from_pretrained(save_directory)

In [12]:
def classify_query(query: str, model, tokenizer) -> str:
    
    # Set the device to GPU if available, otherwise use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    
    # Tokenize the input query
    inputs = tokenizer(query, return_tensors='pt', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get model predictions without computing gradients
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
    
    # Map the predicted class index to the corresponding label
    label_map = {
        0: 'Вопрос по базе знаний',
        1: 'Тех поддержка',
        2: 'Нерелевантный'
    }
    predicted_label = label_map[predicted_class]
    
    return predicted_label

In [13]:
result = classify_query(query, albert_model, albert_tokenizer)
result

'Вопрос по базе знаний'

In [21]:
data = pd.read_csv('albert_data_formatted.csv').drop('Unnamed: 0', axis=1)
data.head()

,query,label,label_id
0,Заголовок статьи,Нерелевантный запрос,2
1,Подписание контракта после отказа от заключени...,Вопросы по базе знаний,0
2,Участие в котировочной сессии,Вопросы по базе знаний,0
3,Страница контракта организации,Вопросы по базе знаний,0
4,Определение победителя и заключение контракта ...,Вопросы по базе знаний,0


In [22]:
test_rag = data[data.label_id == 0][:10]
test_rag

,query,label,label_id
1,Подписание контракта после отказа от заключени...,Вопросы по базе знаний,0
2,Участие в котировочной сессии,Вопросы по базе знаний,0
3,Страница контракта организации,Вопросы по базе знаний,0
4,Определение победителя и заключение контракта ...,Вопросы по базе знаний,0
5,Создание и публикация закупки с неизвестным об...,Вопросы по базе знаний,0
6,Настройка MS Word перед работой,Вопросы по базе знаний,0
7,Новая версия опубликованного шаблона,Вопросы по базе знаний,0
8,Утверждение заявки на регистрацию новой органи...,Вопросы по базе знаний,0
10,Уведомления о наступающих сроках исполнения этапа,Вопросы по базе знаний,0
11,Как заполнить раздел «Статистические коды» при...,Вопросы по базе знаний,0


In [23]:
test_help = data[data.label_id == 1][:10]
test_help

,query,label,label_id
9,Разблокировка пользователя,Техническая поддержка,1
12,При подаче заявки на регистрацию компании возн...,Техническая поддержка,1
13,Уведомление «Есть ограничения»,Техническая поддержка,1
22,"Ошибка сохранения СТЕ ""Не заполнены обязательн...",Техническая поддержка,1
24,"При создании оферты ошибка ""СТЕ находится в ар...",Техническая поддержка,1
34,"В нашей оферте отображается, что по ней заключ...",Техническая поддержка,1
40,Ошибка загрузки файла YML,Техническая поддержка,1
42,"""Ошибка формата"" при загрузке прайс-листа",Техническая поддержка,1
49,"Заявка СТЕ утверждена, оферта отсутствует в ""У...",Техническая поддержка,1
50,"Ошибка ""Некорректное наименование файла""",Техническая поддержка,1


In [24]:
test_irre = data[data.label_id == 2][:10]
test_irre

,query,label,label_id
0,Заголовок статьи,Нерелевантный запрос,2
118,Видео,Нерелевантный запрос,2
134,Карта сайта,Нерелевантный запрос,2
156,Вопросы и ответы,Нерелевантный запрос,2
268,Карта сайта,Нерелевантный запрос,2
372,Тестовая статья,Нерелевантный запрос,2
390,Карта сайта,Нерелевантный запрос,2
459,Новая статья про котировочные сессии 1,Нерелевантный запрос,2
464,тест знания,Нерелевантный запрос,2
465,тест знания,Нерелевантный запрос,2


In [32]:
for item in test_rag['query']:
    print(item, '-', classify_query(item, albert_model, albert_tokenizer))

Подписание контракта после отказа от заключения со стороны заказчика - Вопрос по базе знаний
Участие в котировочной сессии - Вопрос по базе знаний
Страница контракта организации - Вопрос по базе знаний
Определение победителя и заключение контракта с победителем закупки с неизвестным объемом - Вопрос по базе знаний
Создание и публикация закупки с неизвестным объемом (по цене за единицу продукции) - Вопрос по базе знаний
Настройка MS Word перед работой - Вопрос по базе знаний
Новая версия опубликованного шаблона - Вопрос по базе знаний
Утверждение заявки на регистрацию новой организации - Вопрос по базе знаний
Уведомления о наступающих сроках исполнения этапа - Вопрос по базе знаний
Как заполнить раздел «Статистические коды» при подаче заявки на регистрацию компании? - Вопрос по базе знаний


In [33]:
for item in test_help['query']:
    print(item, '-', classify_query(item, albert_model, albert_tokenizer))

Разблокировка пользователя - Вопрос по базе знаний
При подаче заявки на регистрацию компании возникает ошибка «Не заполнены поля «Банковский идентификационный код» и «Расчетный счет» - Вопрос по базе знаний
Уведомление «Есть ограничения» - Вопрос по базе знаний
Ошибка сохранения СТЕ "Не заполнены обязательные характеристики: ..." - Тех поддержка
При создании оферты ошибка "СТЕ находится в архиве" - Тех поддержка
В нашей оферте отображается, что по ней заключены контракты, но у нас контрактов нет - Вопрос по базе знаний
Ошибка загрузки файла YML - Тех поддержка
"Ошибка формата" при загрузке прайс-листа - Вопрос по базе знаний
Заявка СТЕ утверждена, оферта отсутствует в "Управление офертами" - Тех поддержка
Ошибка "Некорректное наименование файла" - Тех поддержка


In [34]:
for item in test_irre['query']:
    print(item, '-', classify_query(item, albert_model, albert_tokenizer))

Заголовок статьи - Вопрос по базе знаний
Видео - Нерелевантный
Карта сайта - Вопрос по базе знаний
Вопросы и ответы - Вопрос по базе знаний
Карта сайта - Вопрос по базе знаний
Тестовая статья - Вопрос по базе знаний
Карта сайта - Вопрос по базе знаний
Новая статья про котировочные сессии 1 - Нерелевантный
тест знания - Вопрос по базе знаний
тест знания - Вопрос по базе знаний


Ну альберт увы, но тут датасет хуйни и домен говна
Хотя бы работает